<a href="https://colab.research.google.com/github/acdc2019/algo-trading/blob/main/python/notebooks/strategies/rsi_strategy/RSIStrategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **RSI Strategy**
## **15 mins strategy**
* Step 1: A 15 min candle has to close above 70 RSI value.
* Step 2: Above High of this candle Buy signal is generated.
* Step 3: Stop Loss of the trade has to be below the earlier 15 min candles low.
* Step 4: Volume of the candle in which buy signal is generated has to be atleast 2 to 3 times of earlier 5 candles (***Q: average of earlier 5 candle volume?***)
* Step 5: Hourly candles RSI should be above 50 for confirmation.
* Step 6: In the last 5 candles the stock movement should not be more than 4% of the day movement
* Step 7: If Stop Loss of the candle is less than 6000 INR then the trade should be executed

#### **Strategy Parameters**
* window_start, window_end: Dates between which to look for signal
* rsi_15min = 70
* rsi_60min = 50
* stop_loss = 6000
* back_candles = 5
* volume_multiple = 2 or 3 times
* daily_movement_pct = 4

Install libs

In [1]:
!pip install ta
!pip install pandas==1.3.5
!pip install plotly
!pip install numpy

  Created wheel for ta: filename=ta-0.9.0-py3-none-any.whl size=28908 sha256=a7f8c77c2755fd6518af55f124edeffd66238656c7398dd77841534a6d3d6aae
  Stored in directory: /root/.cache/pip/wheels/72/78/64/cc1c01506a1010a9845e9bd7c69333730f7174661228ea4f98
Successfully built ta
     |████████████████████████████████| 11.3 MB 5.3 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.3.5 which is incompatible.


In [ ]:
!wget https://raw.githubusercontent.com/acdc2019/algo-trading/main/python/strategy/helpers.py?token=GHSAT0AAAAAABQZJYETEQYUGLNGHAJZQ7CAYPP566Q -O helpers.py

--2022-01-25 12:39:39--  https://raw.githubusercontent.com/acdc2019/algo-trading/main/python/strategy/helpers.py?token=GHSAT0AAAAAABQZJYETEQYUGLNGHAJZQ7CAYPP566Q
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... ^C


Import Libs

In [1]:
import pandas as pd
from ta.momentum import RSIIndicator
from ta.trend import ADXIndicator
from plotly.subplots import make_subplots
from datetime import date
import numpy as np

In [79]:
class Signal():
    def __init__(self, strategy: str, sym: str, lot_size: int, ts: date, entry_price: float, stop_loss: float,
                 rsi: float, hourly_rsi: float, vol: int, mean_vol: int, daily_mov_pct: float) -> None:
        self.strategy = strategy
        self.sym = sym
        self.lot_size = lot_size
        self.ts = ts
        self.entry_price = entry_price
        self.stop_loss = stop_loss
        self.rsi = rsi
        self.hourly_rsi = hourly_rsi
        self.vol = vol
        self.mean_vol = mean_vol
        self.daily_mov_pct = daily_mov_pct
        self.pnl = 0
        self.comment = ""

    def __str__(self) -> str:
        return "Strategy: {}, Sym: {}, TS: {}, Entry: {}, StopLoss: {}, PnL: {}, Comment: {}".format(
            self.strategy, self.sym, self.ts, self.entry_price, self.stop_loss, self.pnl, self.comment)

class BackTestResult():
    def __init__(self, sym: str, signals: list) -> None:
        self.signals = signals
        self.sym = sym
        self.total_pnl = self._calc_total_pnl()
        self.gross_profit = self._calc_gross_profit()
        self.gross_loss = self._calc_gross_loss()
        self.profit_factor = self._calc_profit_factor()

    def _calc_profit_factor(self) -> float:
        gross_profit = self._calc_gross_profit()
        gross_loss = abs(self._calc_gross_loss())
        if (gross_loss == 0):
            gross_loss = 0.1

        profit_factor = gross_profit/gross_loss
        return round(profit_factor, 2)

    def _calc_gross_profit(self) -> float:
        gross_profit = 0
        for signal in self.signals:
            if (signal.pnl > 0):
                gross_profit = gross_profit + signal.pnl

        return round(gross_profit, 2)

    def _calc_gross_loss(self) -> float:
        gross_loss = 0
        for signal in self.signals:
            if (signal.pnl < 0):
                gross_loss = gross_loss - signal.pnl

        return round(gross_loss, 2)

    def _calc_strike_rate(self) -> float:
        if (len(self.signals) == 0):
            return 0

        profit = 0
        notional = 0
        for signal in self.signals:
            if (signal.pnl > 0):
                profit = profit+signal.pnl

            notional = notional + abs(signal.pnl)

        return round(profit/notional, 2)

    def _calc_total_pnl(self) -> float:
        total_pnl = 0
        for signal in self.signals:
            total_pnl = total_pnl + signal.pnl

        return round(total_pnl, 2)

    def __str__(self) -> str:
        return 'Sym,{},Total PnL,{},ProfitFactor,{},GrossProfit,{},GrossLoss,{},Total Signals,{}'.format(
            self.sym, self.total_pnl, self.profit_factor, self.gross_profit, self.gross_loss, len(self.signals))


In [456]:
def get_previous_candles(df: pd.DataFrame, index, n: int, include_index=False):
    '''
    Returns previous n candles from the given index in the DataFrame
    Parameters:
    df (DataFrame): DataFrame from which to return the previous candles
    index (DataFrame Index): DataFrame Index from which to return the previous candles
    n (int): Number of previous candles to return from index
    include_index (bool): If current index should be included in returned DataFrame
    Returns:
    DataFrame: Pandas dataframe with the previous n candles
    '''
    loc = df.index.get_loc(index)
    fromIdx = loc-n
    toIdx = loc+1 if include_index else loc
    return df.iloc[fromIdx:toIdx]


def get_next_candles(df: pd.DataFrame, index, n: int):
    '''
    Returns next n candles from the given index in the DataFrame
    Parameters:
    df (DataFrame): DataFrame from which to return the next candles
    index (DataFrame Index): DataFrame Index from which to return the next candles
    n (int): Number of next candles to return from index
    Returns:
    DataFrame: Pandas dataframe with the next n candles
    '''
    loc = df.index.get_loc(index)
    return df.iloc[loc+1:loc+1+n]


def get_hourly_df(df: pd.DataFrame, index) -> pd.DataFrame:
    df_temp = df[:index]
    # print(index)
    # print(df_15min_temp.tail(10))
    df_60min_o = df_temp['Open'].resample(
        '60Min', offset='30Min').apply({'Open': 'first'})
    df_60min_h = df_temp['High'].resample(
        '60Min', offset='30Min').apply({'High': 'max'})
    df_60min_l = df_temp['Low'].resample(
        '60Min', offset='30Min').apply({'Low': 'min'})
    df_60min_c = df_temp['Close'].resample(
        '60Min', offset='30Min').apply({'Close': 'last'})
    df_60min_vol = df_temp['Volume'].resample(
        '60Min', offset='30Min').apply({'Volume': 'sum'})

    df_60min = pd.concat([df_60min_o, df_60min_h, df_60min_l,
                         df_60min_c, df_60min_vol], axis=1)
    df_60min.dropna(subset=['Open'], inplace=True)

    rsi = RSIIndicator(df_60min['Close']).rsi()
    df_60min = df_60min.assign(rsi=rsi.values)
    return df_60min


def get_tanaji_pct(df: pd.DataFrame, index, n: int, high: float) -> float:
    prev_candles = get_previous_candles(df, index, n)
    min_low = prev_candles['Low'].min()
    daily_movement = high - min_low
    tanaji_pct = (daily_movement/min_low)*100
    return tanaji_pct


def get_sell_tanaji_pct(df: pd.DataFrame, index, n: int, low: float) -> float:
    prev_candles = get_previous_candles(df, index, n)
    max_high = prev_candles['High'].max()
    daily_movement = max_high - low
    tanaji_pct = (daily_movement/max_high)*100
    return tanaji_pct


def generate_buy_signal_pnl(signal: Signal, df: pd.DataFrame, stop_gain: int) -> None:
    df = df[signal.ts:].copy()
    df.drop(index=df.index[0], axis=0, inplace=True)
    rnum = 0
    stop_loss_gap = signal.entry_price - signal.stop_loss
    rsi_breached = False
    adx = 0
    prev_adx = 0
    adx_breached = False

    for index, row in df.iterrows():
        adx = row['adx']
        if (row['Low'] < signal.stop_loss):
            #print('rnum: {}, index: {}, low: {}, sl: {}'.format(rnum, index, row['Low'], signal.stop_loss))
            signal.pnl = -1 * (signal.entry_price -
                               signal.stop_loss) * signal.lot_size
            signal.comment = "Stop Loss Breached at {}".format(index)
            break

        if ((row['High'] - signal.entry_price) * signal.lot_size >= stop_gain):
            signal.pnl = stop_gain
            signal.comment = "Stop Gain reached at {}".format(index)
            break

        if (rsi_breached == True):
            signal.pnl = (row['Open'] - signal.entry_price) * signal.lot_size
            signal.comment = "RSI Breached at {}".format(index)
            break
        
        if (adx_breached == True and prev_adx - adx >= 0.25):
            adx_breached = False
            signal.pnl = (row['Open'] - signal.entry_price) * signal.lot_size
            signal.comment = "ADX Breached at {}".format(index)
            break

        if (prev_adx != 0 and prev_adx - adx >= 0.25 ):
            adx_breached = True

        if (row['High'] - signal.stop_loss > 2*stop_loss_gap):
            # print('New Stoploss: {}, Old Stoploss: {}, Stoploss gap: {}'.format(                row['High'] - stop_loss_gap, signal.stop_loss, stop_loss_gap))
            # signal.stop_loss = row['High'] - stop_loss_gap
            signal.stop_loss = signal.stop_loss

        '''if (row['rsi'] < 70 and row['Volume'] >= 2*row['vol_sma5']):
            rsi_breached = True'''

        if(rnum > 50):
            signal.pnl = (row['Close'] -
                          signal.entry_price) * signal.lot_size
            # print('Signal failed to generate PnL: {}'.format(signal.pnl))
            signal.comment = "Signal failed to generate {}".format(index)
            break

        rnum = rnum+1
        prev_adx = adx


def generate_sell_signal_pnl(signal: Signal, df: pd.DataFrame, stop_gain: float) -> None:
    df = df[signal.ts:].copy()
    df.drop(index=df.index[0], axis=0, inplace=True)
    rnum = 0
    for index, row in df.iterrows():
        if (row['High'] > signal.stop_loss):
            #print('rnum: {}, index: {}, low: {}, sl: {}'.format(rnum, index, row['Low'], signal.stop_loss))
            signal.pnl = (signal.entry_price -
                          signal.stop_loss) * signal.lot_size
            break

        if ((signal.entry_price - row['Low']) * signal.lot_size >= stop_gain):
            signal.pnl = stop_gain
            break

        if(rnum > 50):
            signal.pnl = (signal.entry_price - row['Close']) * signal.lot_size
            # print('Signal failed to generate PnL: {}'.format(signal.pnl))
            break
        rnum = rnum+1

## Load 15 min and 60 min stock data and calculate RSI

In [466]:
file_15min = '/content/BPCL-HIST.csv'

df_15min = pd.read_csv(file_15min, parse_dates=['Date'], index_col=['Date'])

# Add RSI
rsi = RSIIndicator(df_15min['Close']).rsi()
df_15min = df_15min.assign(rsi=rsi.values)

# Add Vol SMA 5
vol_sma5 = df_15min['Volume'].rolling(5).mean()
df_15min = df_15min.assign(vol_sma5=vol_sma5.values)

# Add ADX
ADX = ADXIndicator(high=df_15min['High'], low=df_15min['Low'], close=df_15min['Close'])
df_15min = df_15min.assign(adx=ADX.adx().values)
df_15min = df_15min.assign(adx_neg=ADX.adx_neg().values)
df_15min = df_15min.assign(adx_pos=ADX.adx_pos().values)
df_15min.tail()


/usr/local/lib/python3.7/dist-packages/ta/trend.py:769: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.7/dist-packages/ta/trend.py:774: RuntimeWarning:

invalid value encountered in double_scalars



,Open,High,Low,Close,Volume,rsi,vol_sma5,adx,adx_neg,adx_pos
Date,,,,,,,,,,
2021-12-17 14:15:00+05:30,384.10,385.15,383.50,384.1,193139,44.737142,161437.2,31.157536,21.790546,14.364424
2021-12-17 14:30:00+05:30,384.20,384.70,383.30,384.2,353354,45.261788,210314.0,30.536146,21.348336,13.518024
2021-12-17 14:45:00+05:30,384.20,384.75,383.35,384.4,269735,46.358639,241893.6,29.902653,20.074491,12.924515
2021-12-17 15:00:00+05:30,384.35,385.50,383.80,384.9,943163,49.104396,400915.2,28.493453,18.621469,15.182321
2021-12-17 15:15:00+05:30,384.90,386.60,383.85,385.0,990599,49.659399,549998.0,26.753248,16.536328,17.961283


## Set Strategy Parameters

In [467]:
# Strategy params
strategy = 'RSIBuy'
sym = 'BPCL'
window_start = '2021-01-01 00:00:00'
window_end = '2021-12-15 00:00:00'
rsi_15min = 70
rsi_60min = 50
lot_size = 1800
stop_loss = 6500
stop_gain = 30000
back_candles = 5
adx_low = 30
adx_high = 50
volume_multiple = 2 # 2 or 3 times
daily_movement_pct = 4
check_volume = True


## Actual Strategy Implementation

### Step 1: A 15 min candle has to close above 70 RSI value
Get all the candles in the window that close above required RSI value

In [478]:
curr_window_df = df_15min[window_start:window_end]

rsi_filter = (curr_window_df['rsi'] > rsi_15min)
df = curr_window_df[rsi_filter]

bullish_filter = (df['Close'] > df['Open'])
df = df[bullish_filter]

temp_df = pd.DataFrame(columns=df_15min.columns)
temp_df.index.name = 'Date'
back_candles_df = pd.DataFrame()

for index, row in df.iterrows():
    back_candles_df = get_previous_candles(curr_window_df, index, 1)
    prev_rsi = 0
    if (back_candles_df.shape[0] != 0):
        prev_rsi = back_candles_df.iloc[0]['rsi']

    if (prev_rsi < 70):
        temp_df.loc[index] = row

df = temp_df[df_15min.columns]
df.head()


,Open,High,Low,Close,Volume,rsi,vol_sma5,adx,adx_neg,adx_pos
Date,,,,,,,,,,
2021-01-04 09:15:00+05:30,384.90,389.75,384.90,387.75,1014218.0,71.603561,361549.2,21.059172,10.990129,40.204063
2021-01-04 14:45:00+05:30,392.80,393.80,392.25,393.75,237239.0,70.541073,261798.4,45.435571,6.536958,27.316544
2021-01-08 09:15:00+05:30,395.00,404.25,394.25,403.85,2889215.0,79.677557,790290.6,15.238619,16.091483,46.826549
2021-01-13 09:30:00+05:30,407.05,418.25,406.10,416.25,3218713.0,75.422706,1041190.8,19.918661,15.083159,46.096487
2021-01-14 15:00:00+05:30,422.45,429.70,422.30,427.20,2517693.0,77.983412,994253.4,34.866566,8.413510,41.624960



### Step 4: Volume check

In [479]:
if (check_volume == True):
    temp_df = pd.DataFrame(columns=df_15min.columns)
    temp_df.index.name = 'Date'

    for index, row in df.iterrows():
        # Compare volume against previous mean volume
        if(row['Volume'] > volume_multiple*row['vol_sma5']):
            temp_df.loc[index] = row

    df = temp_df[df_15min.columns]
    df.head()

### Step 5: Hourly candle RSI check

In [480]:
temp_df = pd.DataFrame(columns=df.columns)
temp_df.index.name = 'Date'

for index, row in df.iterrows():
    df_60min = get_hourly_df(df_15min, index)
    # print(df_60min.tail(15))

    rsi = df_60min.iloc[-1]['rsi']
    if(rsi > rsi_60min):
        # This 15min candle is eligible for signal
        # print('Hourly candle RSI is greater than 50..', rsi)
        temp_df.loc[index] = row

df = temp_df
df.head()


,Open,High,Low,Close,Volume,rsi,vol_sma5,adx,adx_neg,adx_pos
Date,,,,,,,,,,
2021-01-04 09:15:00+05:30,384.90,389.75,384.90,387.75,1014218.0,71.603561,361549.2,21.059172,10.990129,40.204063
2021-01-08 09:15:00+05:30,395.00,404.25,394.25,403.85,2889215.0,79.677557,790290.6,15.238619,16.091483,46.826549
2021-01-13 09:30:00+05:30,407.05,418.25,406.10,416.25,3218713.0,75.422706,1041190.8,19.918661,15.083159,46.096487
2021-01-14 15:00:00+05:30,422.45,429.70,422.30,427.20,2517693.0,77.983412,994253.4,34.866566,8.413510,41.624960
2021-01-21 10:30:00+05:30,416.55,421.55,416.20,421.50,1200951.0,71.892306,490104.4,20.259904,13.745139,38.545845


### Step 6: Stock movement check

In [481]:
temp_df = pd.DataFrame(columns=df.columns)
temp_df.index.name = 'Date'

for index, row in df.iterrows():
    tanaji_pct = get_tanaji_pct(df_15min, index, 5, row['High'])
    # print('tanaji_pct:', tanaji_pct)
    if(tanaji_pct < daily_movement_pct):
        temp_df.loc[index] = row

df = temp_df
df.head()

,Open,High,Low,Close,Volume,rsi,vol_sma5,adx,adx_neg,adx_pos
Date,,,,,,,,,,
2021-01-04 09:15:00+05:30,384.90,389.75,384.90,387.75,1014218.0,71.603561,361549.2,21.059172,10.990129,40.204063
2021-01-08 09:15:00+05:30,395.00,404.25,394.25,403.85,2889215.0,79.677557,790290.6,15.238619,16.091483,46.826549
2021-01-14 15:00:00+05:30,422.45,429.70,422.30,427.20,2517693.0,77.983412,994253.4,34.866566,8.413510,41.624960
2021-01-21 10:30:00+05:30,416.55,421.55,416.20,421.50,1200951.0,71.892306,490104.4,20.259904,13.745139,38.545845
2021-02-02 14:00:00+05:30,405.10,411.00,405.10,410.85,758797.0,72.439541,263513.2,32.738867,10.975324,39.039180


### Step 7: Stop Loss check

In [482]:
temp_df = pd.DataFrame(columns=df.columns)
temp_df.index.name = 'Date'

for index, row in df.iterrows():
    prev_candle = get_previous_candles(df_15min, index, 1)
    prev_low = prev_candle.iloc[0]['Low']
    if((row['High'] - prev_low)*lot_size > stop_loss):
        print(index, 'Stop Loss greater than 6000 INR. Do not trade', (row['High'] - prev_low)*lot_size)
        continue
    else:
        print(index, 'Stop Loss within range', (row['High'] - prev_low)*lot_size)
        temp_df.loc[index] = row

df = temp_df
df

2021-01-04 09:15:00+05:30 Stop Loss greater than 6000 INR. Do not trade 14940.00000000002
2021-01-08 09:15:00+05:30 Stop Loss greater than 6000 INR. Do not trade 22050.0
2021-01-14 15:00:00+05:30 Stop Loss greater than 6000 INR. Do not trade 15659.99999999998
2021-01-21 10:30:00+05:30 Stop Loss greater than 6000 INR. Do not trade 14490.00000000002
2021-02-02 14:00:00+05:30 Stop Loss greater than 6000 INR. Do not trade 12869.99999999996
2021-02-18 14:00:00+05:30 Stop Loss greater than 6000 INR. Do not trade 17100.0
2021-02-19 12:00:00+05:30 Stop Loss greater than 6000 INR. Do not trade 20519.99999999996
2021-03-30 15:15:00+05:30 Stop Loss greater than 6000 INR. Do not trade 14579.999999999938
2021-04-01 15:00:00+05:30 Stop Loss greater than 6000 INR. Do not trade 13050.0
2021-05-21 09:15:00+05:30 Stop Loss greater than 6000 INR. Do not trade 26640.000000000022
2021-05-24 14:45:00+05:30 Stop Loss greater than 6000 INR. Do not trade 7559.99999999998
2021-06-03 09:15:00+05:30 Stop Loss gre

,Open,High,Low,Close,Volume,rsi,vol_sma5,adx,adx_neg,adx_pos
Date,,,,,,,,,,
2021-06-04 10:30:00+05:30,477.05,478.60,476.50,478.60,493353.0,70.360389,230340.8,24.944911,12.891258,32.157489
2021-08-26 14:15:00+05:30,461.65,464.40,461.40,464.40,398924.0,72.088122,198039.0,20.369783,9.977204,32.178473
2021-09-13 15:15:00+05:30,497.20,499.25,497.20,498.90,1299335.0,73.082868,638475.2,45.421197,7.171736,36.240082
2021-09-29 15:15:00+05:30,436.80,438.00,436.75,437.85,1134558.0,72.427542,507885.4,22.862885,13.820983,33.007951
2021-10-13 11:00:00+05:30,457.30,458.90,456.65,458.35,889941.0,70.021283,360439.2,29.774818,14.918668,22.547209


In [486]:
# ADX Check
temp_df = pd.DataFrame(columns=df.columns)
temp_df.index.name = 'Date'

for index, row in df.iterrows():
    back_candles_df = get_previous_candles(curr_window_df, index, 1)
    prev_adx = back_candles_df.iloc[0]['adx']
    if (row['adx'] >= adx_low and row['adx'] <= adx_high):
        temp_df.loc[index] = row

df = temp_df
df

,Open,High,Low,Close,Volume,rsi,vol_sma5,adx,adx_neg,adx_pos
Date,,,,,,,,,,
2021-09-13 15:15:00+05:30,497.2,499.25,497.2,498.9,1299335.0,73.082868,638475.2,45.421197,7.171736,36.240082


## **Final Signal**

In [484]:
signals = list()
for index, row in df.iterrows():
    stop_loss_candle = get_previous_candles(df_15min, index, 1)
    sig_stop_loss = stop_loss_candle.iloc[0]['Low']
    sig_entry = row['High']
    sig_daily_mov_pct = np.round(get_tanaji_pct(df_15min, index, 5, row['High']), 2)
    sig_rsi = np.round(row['rsi'], 2)
    sig_hourly_rsi = np.round(get_hourly_df(df_15min, index).iloc[-1]['rsi'], 2)
    sig_mean_volume = row['vol_sma5']
    sig_volume = row['Volume']
    signal = Signal(strategy=strategy, sym=sym, lot_size=lot_size,
                    ts=index, entry_price=sig_entry, stop_loss=sig_stop_loss,
                    rsi=sig_rsi, hourly_rsi=sig_hourly_rsi, vol=sig_volume,
                    mean_vol=sig_mean_volume, daily_mov_pct=sig_daily_mov_pct)
    signals.append(signal)

for signal in signals:
    generate_buy_signal_pnl(signal, df_15min, stop_gain=stop_gain)

btResult = BackTestResult(sym=sym, signals=signals)

for signal in btResult.signals:
    print(signal)

print(btResult)

Strategy: RSIBuy, Sym: BPCL, TS: 2021-09-13 15:15:00+05:30, Entry: 499.25, StopLoss: 496.1, PnL: 269.9999999999591, Comment: ADX Breached at 2021-09-14 12:00:00+05:30
Sym,BPCL,Total PnL,270.0,ProfitFactor,2700.0,GrossProfit,270.0,GrossLoss,0,Total Signals,1


In [485]:
import plotly.graph_objects as go
import plotly.offline as py

back=5
next=50
sig_param_col1 = ['Time','Entry INR','StopLoss INR','Daily Mov %','Volume','Mean Vol','15Min RSI','Hourly RSI','PnL','Comments']
for signal in signals:
  prev_candles = get_previous_candles(df_15min, signal.ts, back, True)
  next_candles = get_next_candles(df_15min, signal.ts, next)
  
  candles = pd.concat([prev_candles, next_candles])
  candles['DateStr'] = candles.index.strftime('%d-%m %H:%M')

  # Get all strategy params for this signal

  sig_stop_loss = signal.stop_loss
  sig_entry = signal.entry_price
  sig_daily_mov_pct = signal.daily_mov_pct
  sig_rsi = signal.rsi
  sig_hourly_rsi = signal.hourly_rsi
  sig_mean_volume = signal.mean_vol
  sig_volume = signal.vol

  fig = make_subplots(rows=3, cols=2, shared_xaxes=False,
               subplot_titles=('OHLC', 'Signal Params', 'Volume & RSI','', 'ADX', ''), 
               vertical_spacing=0.1, 
               horizontal_spacing=0.01,
               row_width=[0.25, 0.25, 0.5],
               column_widths=[0.8,0.2],
               specs=[[{"secondary_y": False, "type": "candlestick"},{"secondary_y": False, "type":"table"}], 
                      [{"secondary_y": True}, {"secondary_y": False}],
                      [{"secondary_y": True}, {"secondary_y": False}]])
    
  fig.add_trace(go.Candlestick(x=candles['DateStr'],
                     open=candles['Open'],
                     high=candles['High'],
                     low=candles['Low'],
                     close=candles['Close'],
                     name='Signal Chart',
                     increasing_line_color='yellow',
                     increasing_fillcolor='yellow',
                     decreasing_line_color='red',
                     decreasing_fillcolor='red',),
                     row=1,col=1)
  
  fig.add_annotation(x=back,y=sig_entry,
                     text='Signal')
  
  # Position Entry Point
  fig.add_shape(type='line', 
                x0=-1,x1=back+next+2,
                y0=sig_entry, y1=sig_entry, 
                line=dict(color='Green'),
                row=1,col=1)
  
  # Position Stop Loss
  fig.add_shape(type='line', 
                x0=-1,x1=back+next+2,
                y0=sig_stop_loss, y1=sig_stop_loss, 
                line=dict(color='Red'),
                row=1,col=1)
  
  fig.add_annotation(x=back,y=sig_rsi,
                      text='Signal',row=2, col=1)
  
  fig.add_shape(type='line', x0=-1,x1=back+next+2,y0=rsi_15min, y1=rsi_15min, 
                line=dict(color='Green'),
                row=2,col=1)

  # Signal Parameters Table
  fig.add_trace(go.Table(header=dict(values=['Param','Value'],
                                     line_color='white',
                                     fill_color='darkslategray',
                                     align='left'),
                         cells=dict(values=[sig_param_col1,
                                           [signal.ts.strftime('%d-%m %H:%M'), sig_entry, 
                                            sig_stop_loss, sig_daily_mov_pct, sig_volume, 
                                            sig_mean_volume, sig_rsi, sig_hourly_rsi, signal.pnl, signal.comment]],
                                    line_color='white',
                                    fill_color='black',
                                    align='left')),
                row=1, col=2)

  # RSI
  fig.add_trace(go.Scatter(x=candles['DateStr'], y=candles['rsi'], name='rsi',            
           marker_color='Cyan'),
           row=2, col=1)
  
  # Volume
  fig.add_trace(go.Bar(x=candles['DateStr'], y=candles['Volume'], name='Volume',            
           marker_color='rgb(55, 55, 109)',
           width=np.array([0.5]*candles.size)),
           secondary_y=True,
          row=2, col=1)
  
  # ADX
  fig.add_trace(go.Scatter(x=candles['DateStr'], y=candles['adx'], name='adx',            
           marker_color='Cyan'),
           row=3, col=1)
  fig.add_trace(go.Scatter(x=candles['DateStr'], y=candles['adx_neg'], name='adx_neg',            
           marker_color='Blue'),
           row=3, col=1)
  fig.add_trace(go.Scatter(x=candles['DateStr'], y=candles['adx_pos'], name='adx_pos',            
           marker_color='Yellow'),
           row=3, col=1)

  fig.update_xaxes(type='category', rangeslider=dict(visible=False))
  fig.update_xaxes(showgrid=False, nticks=5)
  fig.update_yaxes(showgrid=False)
  fig.update_layout(
    title='Signal generated for RSI 15mins Strategy',
    title_x = 0.5,
    autosize=False,
    width=1450,
    height=850,
    plot_bgcolor='rgb(5,5,5)',
    paper_bgcolor='rgb(0,0,0)',
    font_color='white')

  py.iplot(fig)
  